# ResNet CIFAR-10 with tensorboard

This notebook shows how to use TensorBoard, and how the training job writes checkpoints to a external bucket.
The model used for this notebook is a ResNet model, trained with the CIFAR-10 dataset.
See the following papers for more background:

[Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Dec 2015.

[Identity Mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Jul 2016.

### Set up the environment

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### Download the CIFAR-10 dataset
Downloading the test and training data will take around 5 minutes.

In [2]:
import utils

utils.cifar10_download()

cifar dataset already downloaded


### Upload the data to a S3 bucket

In [3]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/DEMO-cifar10')

**sagemaker_session.upload_data** will upload the CIFAR-10 dataset from your machine to a bucket named **sagemaker-{region}-{*your aws account number*}**, if you don't have this bucket yet, sagemaker_session will create it for you.

### Complete source code
- [source_dir/resnet_model.py](source_dir/resnet_model.py): ResNet model
- [source_dir/resnet_cifar_10.py](source_dir/resnet_cifar_10.py): main script used for training and hosting

## Create a training job using the sagemaker.TensorFlow estimator

In [4]:
from sagemaker.tensorflow import TensorFlow


source_dir = os.path.join(os.getcwd(), 'source_dir')
estimator = TensorFlow(entry_point='resnet_cifar_10.py',
                       source_dir=source_dir,
                       role=role,
                       framework_version='1.12.0',
                       hyperparameters={'throttle_secs': 30},
                       training_steps=1000, evaluation_steps=100,
                       train_instance_count=2, train_instance_type='ml.c4.xlarge', 
                       base_job_name='tensorboard-example')

estimator.fit(inputs, run_tensorboard_locally=True)

No handlers could be found for logger "sagemaker"


2020-05-06 22:10:41 Starting - Starting the training job...
2020-05-06 22:10:43 Starting - Launching requested ML instances......
2020-05-06 22:11:49 Starting - Preparing the instances for training......
2020-05-06 22:13:00 Downloading - Downloading input data...
2020-05-06 22:13:38 Training - Training image download completed. Training in progress..2020-05-06 22:13:36,861 INFO - root - running container entrypoint
2020-05-06 22:13:36,861 INFO - root - starting train task
2020-05-06 22:13:36,875 INFO - container_support.training - Training starting
2020-05-06 22:13:39,421 INFO - root - running container entrypoint
2020-05-06 22:13:39,422 INFO - root - starting train task
2020-05-06 22:13:39,434 INFO - container_support.training - Training starting
2020-05-06 22:13:44,885 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2020-05-06 22:13:44,885 INFO - tf_container - {"environment": "cloud", "cluster": {"worker": ["algo-2:2222"], "ps": ["algo-1:2223", "algo-

The **```fit```** method will create a training job named **```tensorboard-example-{unique identifier}```** in two **ml.c4.xlarge** instances. These instances will write checkpoints to the s3 bucket **```sagemaker-{your aws account number}```**.

If you don't have this bucket yet, **```sagemaker_session```** will create it for you. These checkpoints can be used for restoring the training job, and to analyze training job metrics using **TensorBoard**. 

The parameter **```run_tensorboard_locally=True```** will run **TensorBoard** in the machine that this notebook is running. Everytime a new checkpoint is created by the training job in the S3 bucket, **```fit```** will download the checkpoint to the temp folder that **TensorBoard** is pointing to.

When the **```fit```** method starts the training, it will log the port that **TensorBoard** is using to display the metrics. The default port is **6006**, but another port can be choosen depending on its availability. The port number will increase until finds an available port. After that the port number will printed in stdout.

It takes a few minutes to provision containers and start the training job.**TensorBoard** will start to display metrics shortly after that.

You can access **TensorBoard** locally at [http://localhost:6006](http://localhost:6006) or using your SageMaker notebook instance [proxy/6006/](/proxy/6006/)(TensorBoard will not work if forget to put the slash, '/', in end of the url). If TensorBoard started on a different port, adjust these URLs to match.This example uses the optional hyperparameter **```throttle_secs```** to generate training evaluations more often, allowing to visualize **TensorBoard** scalar data faster. You can find the available optional hyperparameters [here](https://github.com/aws/sagemaker-python-sdk#optional-hyperparameters).

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [5]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------!

# Make a prediction with fake data to verify the endpoint is up

Prediction is not the focus of this notebook, so to verify the endpoint's functionality, we'll simply generate random data in the correct shape and make a prediction.

In [6]:
import numpy as np

random_image_data = np.random.rand(32, 32, 3)
predictor.predict(random_image_data)

{'model_spec': {'name': u'generic_model',
  'signature_name': u'serving_default',
  'version': {'value': 1588803705L}},
 'outputs': {u'classes': {'dtype': 9,
   'int64_val': [6L],
   'tensor_shape': {'dim': [{'size': 1L}]}},
  u'probabilities': {'dtype': 1,
   'float_val': [0.0044905380345880985,
    0.00014152491348795593,
    0.015392323955893517,
    0.0027393086347728968,
    0.0008531171479262412,
    1.7411322914995253e-05,
    0.7100078463554382,
    3.045869743800722e-05,
    0.2637484669685364,
    0.0025790617801249027],
   'tensor_shape': {'dim': [{'size': 1L}, {'size': 10L}]}}}}

# Cleaning up
To avoid incurring charges to your AWS account for the resources used in this tutorial you need to delete the **SageMaker Endpoint:**

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)